## **Model predicting Urban Heat Island**

In [29]:
import numpy as np
import pandas as pd

In [30]:
uhi = pd.read_csv(r"C:\Users\levie\OneDrive\Desktop\Year 5\project\dataset\UHI_training_data.csv")

In [31]:
uhi.head()

,system:index,EMM,FV,LST,Map,NDVI,class,suhi,.geo
0,1_1_1_1_1_1_1_1_1_2_7_0,0.988258,0.564607,30.783192,50,0.191956,10,1.780245,"{""type"":""MultiPoint"",""coordinates"":[]}"
1,1_1_1_1_1_1_1_1_1_2_21_0,0.987213,0.303351,31.487080,50,0.101995,10,2.484133,"{""type"":""MultiPoint"",""coordinates"":[]}"
2,1_1_1_1_1_1_1_1_1_2_27_0,0.987232,0.308082,31.362094,50,0.103624,10,2.359147,"{""type"":""MultiPoint"",""coordinates"":[]}"
3,1_1_1_1_1_1_1_1_1_2_32_0,0.987747,0.436773,30.672953,50,0.147938,10,1.670006,"{""type"":""MultiPoint"",""coordinates"":[]}"
4,1_1_1_1_1_1_1_1_1_2_41_0,0.989417,0.854355,28.460604,50,0.291729,10,-0.542343,"{""type"":""MultiPoint"",""coordinates"":[]}"


In [32]:
uhi.isnull().sum()

system:index    0
EMM             0
FV              0
LST             0
Map             0
NDVI            0
class           0
suhi            0
.geo            0
dtype: int64

In [33]:
uhi.dtypes

system:index     object
EMM             float64
FV              float64
LST             float64
Map               int64
NDVI            float64
class             int64
suhi            float64
.geo             object
dtype: object

In [34]:
uhi = uhi.drop(['system:index','Map','.geo'],axis = 1)

In [35]:
#Training features for SUHI
independent_features = uhi.drop(['suhi'],axis=1)
dependent_features = uhi['suhi']

#Training features for LULC
features = uhi.drop(['class'],axis=1)
targets = uhi['class']
print(independent_features.shape)

(1540, 5)


We use RandomForestClasify to classify Land Use Land Cover<br>
We use RandomForestRegressor to predict UHI


In [36]:
#Splitting the dataets
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(independent_features,dependent_features,test_size=0.3,random_state=100)
X_train_class,X_test_class,y_train_class,y_test_class = train_test_split(features,targets,test_size=0.3,random_state=100)

In [37]:
# Standardize the values for the regressor
from sklearn.preprocessing import StandardScaler,MinMaxScaler
scale = StandardScaler()
X_train_scaled = scale.fit_transform(X_train)
X_test_scaled = scale.fit_transform(X_test)

#Standardize values of the clasifier
X_train_scaled1 = scale.fit_transform(X_train_class)
X_test_scaled1 = scale.fit_transform(X_test_class)

In [38]:
#Fitting the model for training
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor()

In [39]:
#Random forest regressor-UHI prediction
rfr.fit(X_train_scaled,y_train)
y_pred = rfr.predict(X_test_scaled)

In [40]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
# Evaluating model performance
def model_perform (y_p,y_t):
    mae = mean_absolute_error(y_p,y_t)
    mse = mean_squared_error(y_p,y_t)
    r2 = r2_score(y_p,y_t)
    print(f"The mean absolute error is:{mae:.3f}")
    print(f"The mean squared error is:{mse:.3f}")
    print(f"R2:{r2:.3f}")
model_eval = model_perform(y_pred,y_test)
print(model_eval)

The mean absolute error is:0.023
The mean squared error is:0.001
R2:0.999
None


### Saving the model


In [41]:
import pickle
filename = "UHI_model.sav"
pickle.dump(rfr, open(filename, 'wb'))

In [42]:
import joblib

In [43]:
def load_model():
    return joblib.load("C:/Users/levie/OneDrive/Desktop/Year 5/project/scripts/UHI_model.sav")

model = load_model()

Converting the geojson to lattitude and longitude

In [44]:
uhi.dtypes

EMM      float64
FV       float64
LST      float64
NDVI     float64
class      int64
suhi     float64
dtype: object

In [46]:

import ast
# Convert string to dictionary
uhi_data = pd.read_csv(r"C:\Users\levie\OneDrive\Desktop\Year 5\project\dataset\UHI_training_data.csv")
uhi_data[".geo"] = uhi_data["GeoJSON"].apply(lambda x: ast.literal_eval(x)["coordinates"][0])

# Check if '.geo' column exists
if  '.geo' in uhi_data.columns:
    # Convert '.geo' JSON string to dictionary and extract coordinates
    uhi_data[["Longitude", "Latitude"]] = uhi_data[".geo"].apply(
        lambda x: pd.Series(ast.literal_eval(x)["coordinates"][0])
    )
else:
    print("Column '.geo' not found in dataset!")

# Display the updated dataframe
print(uhi_data.head())

KeyError: 'GeoJSON'

In [26]:
print(uhi_data.columns)

Index(['system:index', 'EMM', 'FV', 'LST', 'Map', 'NDVI', 'class', 'suhi',
       '.geo'],
      dtype='object')
